In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Device configuration
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print('Using device:', device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))

Using device: cpu


In [3]:
# Hyper-parameters
num_epochs = 5
batch_size = 4
learning_rate = 0.001

In [4]:
# Transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # CIFAR has images of shape (3, 32, 32) and values in the range [0, 1] we normalize to [-1, 1]

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data',
                                             train=True,
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data',
                                            train=False,
                                            transform=transform)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                             batch_size=batch_size,
                                             shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                            batch_size=batch_size,
                                            shuffle=False)

labels = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170498071/170498071 [00:56<00:00, 3012239.19it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data


In [6]:
# Convolutional neural network
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = ConvNet().to(device)

In [7]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [8]:
# Training loop
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/5], Step [2000/12500], Loss: 2.2883
Epoch [1/5], Step [4000/12500], Loss: 2.3173
Epoch [1/5], Step [6000/12500], Loss: 2.3016
Epoch [1/5], Step [8000/12500], Loss: 2.3048
Epoch [1/5], Step [10000/12500], Loss: 2.3058
Epoch [1/5], Step [12000/12500], Loss: 2.2739
Epoch [2/5], Step [2000/12500], Loss: 1.8480
Epoch [2/5], Step [4000/12500], Loss: 1.7706
Epoch [2/5], Step [6000/12500], Loss: 2.0886
Epoch [2/5], Step [8000/12500], Loss: 1.6789
Epoch [2/5], Step [10000/12500], Loss: 1.6741
Epoch [2/5], Step [12000/12500], Loss: 2.0716
Epoch [3/5], Step [2000/12500], Loss: 2.1803
Epoch [3/5], Step [4000/12500], Loss: 1.8209
Epoch [3/5], Step [6000/12500], Loss: 1.3429
Epoch [3/5], Step [8000/12500], Loss: 2.0556
Epoch [3/5], Step [10000/12500], Loss: 1.4889
Epoch [3/5], Step [12000/12500], Loss: 2.0505
Epoch [4/5], Step [2000/12500], Loss: 1.8234
Epoch [4/5], Step [4000/12500], Loss: 2.6392
Epoch [4/5], Step [6000/12500], Loss: 1.1738
Epoch [4/5], Step [8000/12500], Loss: 2.0209
Epoc

In [9]:
# Test the model
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
            
    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

Accuracy of the network: 49.24 %
